# PE 파일 분석 가이드 (초보자용)

이 노트북은 `pefile` 라이브러리를 사용하여 PE(Portable Executable) 파일(Windows 실행 파일 .exe, .dll 등)의 구조를 분석하는 방법을 단계별로 설명합니다.

PE 파일 구조를 처음 접하는 분들을 위해 각 단계가 무엇을 의미하는지 간단히 설명합니다.

In [4]:
# 필요한 라이브러리 설치
import sys
!pip install pefile

In [5]:
import pefile
import os

# 분석할 PE 파일의 경로를 설정해주세요.
# macOS나 Linux를 사용 중이라면, 분석할 윈도우용 .exe 파일을 이 폴더에 넣고 경로를 변경해주세요.
pe_path = 'ZoomInstaller.exe' 

if not os.path.exists(pe_path):
    print(f"오류: '{pe_path}' 파일을 찾을 수 없습니다. 분석할 PE 파일 경로를 올바르게 지정해주세요.")
else:
    print(f"분석 대상 파일: {pe_path}")

분석 대상 파일: ZoomInstaller.exe


## 1. PE 파일 로드하기

`pefile.PE()` 함수를 사용하여 파일을 파싱(해석)합니다. 이 과정에서 파일의 헤더와 섹션 정보들이 메모리에 로드됩니다.

In [6]:
try:
    pe = pefile.PE(pe_path)
    print("PE 파일 로드 성공!")
except FileNotFoundError:
    print("파일을 찾을 수 없습니다. pe_path 변수를 확인해주세요.")
except Exception as e:
    print(f"PE 파일 로드 실패: {e}")

PE 파일 로드 성공!


## 2. DOS Header 확인하기

모든 PE 파일은 **DOS Header**로 시작합니다. 이는 아주 옛날 MS-DOS 호환성을 위해 존재하는 부분입니다.
가장 중요한 필드는 `e_magic` (매직 넘버, 'MZ'로 시작)과 `e_lfanew` (NT Header의 위치)입니다.

In [7]:
if 'pe' in locals():
    print(f"[DOS Header]")
    print(f"e_magic: {hex(pe.DOS_HEADER.e_magic)} (Should be 0x5A4D 'MZ')")
    print(f"e_lfanew: {hex(pe.DOS_HEADER.e_lfanew)} (Offset to NT Headers)")
    
    # DOS Header의 전체 구조를 보고 싶다면:
    # print(pe.DOS_HEADER)

[DOS Header]
e_magic: 0x5a4d (Should be 0x5A4D 'MZ')
e_lfanew: 0x118 (Offset to NT Headers)


## 3. NT Headers 확인하기

실제 PE 파일의 핵심 정보가 담긴 곳입니다. 서명(Signature), 파일 헤더(File Header), 옵셔널 헤더(Optional Header)로 구성됩니다.

- **File Header**: 파일의 기계적 특성 (CPU 아키텍처, 섹션 개수 등)
- **Optional Header**: 실행에 필요한 상세 정보 (진입점 주소 Entry Point, 이미지 베이스 등)

In [8]:
if 'pe' in locals():
    print(f"[NT Headers]")
    print(f"Signature: {hex(pe.NT_HEADERS.Signature)} (Should be 0x4550 'PE\\0\\0')")
    
    print("-" * 20)
    print(f"[File Header]")
    print(f"Machine: {hex(pe.FILE_HEADER.Machine)}")
    print(f"Number of Sections: {pe.FILE_HEADER.NumberOfSections}")
    print(f"TimeDateStamp: {pe.FILE_HEADER.TimeDateStamp}")
    
    print("-" * 20)
    print(f"[Optional Header]")
    print(f"Magic: {hex(pe.OPTIONAL_HEADER.Magic)} (0x10B: 32bit, 0x20B: 64bit)")
    print(f"AddressOfEntryPoint: {hex(pe.OPTIONAL_HEADER.AddressOfEntryPoint)}")
    print(f"ImageBase: {hex(pe.OPTIONAL_HEADER.ImageBase)}")

[NT Headers]
Signature: 0x4550 (Should be 0x4550 'PE\0\0')
--------------------
[File Header]
Machine: 0x8664
Number of Sections: 8
TimeDateStamp: 1764243669
--------------------
[Optional Header]
Magic: 0x20b (0x10B: 32bit, 0x20B: 64bit)
AddressOfEntryPoint: 0x22120
ImageBase: 0x140000000


## 4. Sections (섹션) 정보 확인

코드는 `.text`, 데이터는 `.data`, 리소스는 `.rsrc` 같은 이름의 섹션에 나뉘어 저장됩니다.
각 섹션의 이름, 가상 주소(메모리에 로드될 주소), 파일 내에서의 크기 등을 확인할 수 있습니다.

In [17]:
if 'pe' in locals():
    print(f"{'Name':<10} {'Virtual Address':<20} {'Raw Size':<15} {'Characteristics'}")
    print("-" * 60)
    
    for section in pe.sections:
        # 섹션 이름은 바이트로 되어 있어서 디코딩이 필요할 수 있습니다.
        name = section.Name.decode('utf-8', errors='ignore').strip('\x00')
        e = section.get_entropy()
        print(f"{name:<10} 엔트로피: {e:.2f}")
        
        print(f"{name:<10} {hex(section.VirtualAddress):<20} {hex(section.SizeOfRawData):<15} {hex(section.Characteristics)}")

Name       Virtual Address      Raw Size        Characteristics
------------------------------------------------------------
.text      엔트로피: 6.41
.text      0x1000               0x2d000         0x60000020
.rdata     엔트로피: 4.64
.rdata     0x2e000              0x11800         0x40000040
.data      엔트로피: 3.53
.data      0x40000              0x1a00          0xc0000040
.pdata     엔트로피: 5.38
.pdata     0x46000              0x2c00          0x40000040
.fptable   엔트로피: 0.00
.fptable   0x49000              0x200           0xc0000040
_RDATA     엔트로피: 2.44
_RDATA     0x4a000              0x200           0x40000040
.rsrc      엔트로피: 4.90
.rsrc      0x4b000              0xb000          0x40000040
.reloc     엔트로피: 5.28
.reloc     0x56000              0xc00           0x42000040


## 5. Import Table (임포트 함수) 확인

이 실행 파일이 어떤 Windows API(DLL)들을 가져다 쓰는지 확인합니다. 악성코드 분석이나 프로그램 기능을 유추할 때 가장 중요한 부분 중 하나입니다.

In [10]:
if 'pe' in locals() and hasattr(pe, 'DIRECTORY_ENTRY_IMPORT'):
    print("[Imported DLLs & Functions]")
    for entry in pe.DIRECTORY_ENTRY_IMPORT:
        dll_name = entry.dll.decode('utf-8', errors='ignore')
        print(f"\nDLL: {dll_name}")
        
        for imp in entry.imports:
            func_name = imp.name.decode('utf-8', errors='ignore') if imp.name else "Ord(" + str(imp.ordinal) + ")"
            print(f"\t- {func_name} (Address: {hex(imp.address)})")
else:
    print("Import Table이 없거나 파싱되지 않았습니다.")

[Imported DLLs & Functions]

DLL: KERNEL32.dll
	- VirtualFree (Address: 0x14002e000)
	- VirtualAlloc (Address: 0x14002e008)
	- IsProcessorFeaturePresent (Address: 0x14002e010)
	- GetSystemDirectoryW (Address: 0x14002e018)
	- GetModuleHandleA (Address: 0x14002e020)
	- GetVersion (Address: 0x14002e028)
	- GetProcAddress (Address: 0x14002e030)
	- LoadLibraryExW (Address: 0x14002e038)
	- EnterCriticalSection (Address: 0x14002e040)
	- LeaveCriticalSection (Address: 0x14002e048)
	- DeleteCriticalSection (Address: 0x14002e050)
	- ReleaseSemaphore (Address: 0x14002e058)
	- InitializeCriticalSection (Address: 0x14002e060)
	- WaitForSingleObject (Address: 0x14002e068)
	- GetLastError (Address: 0x14002e070)
	- SetEvent (Address: 0x14002e078)
	- CloseHandle (Address: 0x14002e080)
	- ResetEvent (Address: 0x14002e088)
	- CreateSemaphoreA (Address: 0x14002e090)
	- CreateEventA (Address: 0x14002e098)
	- WideCharToMultiByte (Address: 0x14002e0a0)
	- MultiByteToWideChar (Address: 0x14002e0a8)
	- FreeLib

## 마무리

이렇게 `pefile`을 사용하면 PE 파일의 주요 구조를 쉽게 파싱하고 분석할 수 있습니다.
이후에는 다음과 같은 심화 분석도 가능합니다:
- **Export Table 확인**: DLL 파일이 제공하는 함수 목록 확인
- **Resource Section 분석**: 아이콘, 메뉴, 문자열 등 리소스 추출
- **Packer 탐지**: 파일이 압축되거나 암호화되었는지(패킹) 확인

이 노트북을 통해 PE 파일 구조분석의 기초를 다지시길 바랍니다!

## MalConv2 Model Execution
This section demonstrates how to load and run the MalConv2 model using the instructions and files from the `MalConv2-main` directory. Code is based on the README instructions.

In [8]:
import sys
import os
import torch
import numpy as np

def manipulate_exe(input_path, output_path=None, padding_size=0, mode='zero'):
    """
    EXE 파일을 바이너리로 읽어서 패딩을 추가하거나 원본 그대로 저장하는 함수
    
    :param input_path: 원본 exe 파일 경로
    :param output_path: 저장할 경로 (None이면 저장 안 함)
    :param padding_size: 추가할 패딩 크기 (바이트 단위, 0이면 아무것도 안 함)
    :param mode: 'zero' (0으로 채움), 'random' (랜덤 값), 'goodware' (정상 파일 일부 - 구현 필요)
    :return: 조작된 바이너리 데이터 (bytes)
    """
    
    # 1. [핵심] 파일을 바이너리 읽기 모드('rb')로 염
    try:
        with open(input_path, 'rb') as f:
            binary_data = f.read() # 파일 전체를 바이트(bytes)로 읽어옴
            print(f"[Info] {input_path} 로드 완료. 크기: {len(binary_data)} bytes")
    except FileNotFoundError:
        print(f"[Error] 파일을 찾을 수 없습니다: {input_path}")
        return None

    # 2. 패딩 추가 로직 (padding_size가 0이면 이 부분 건너뜀 -> 아무것도 안 함)
    if padding_size > 0:
        print(f"[Processing] 패딩 {padding_size} bytes 추가 중... ({mode} 모드)")
        
        if mode == 'zero':
            # 0x00 (Null Byte)으로 꽉 채움
            padding = b'\x00' * padding_size
            
        elif mode == 'random':
            # 완전 무작위 값으로 채움 (os.urandom 사용)
            padding = os.urandom(padding_size)
            
        else:
            # 기본은 0으로
            padding = b'\x00' * padding_size

        # ★ 바이너리 결합: 원본 뒤에 패딩을 붙임
        binary_data = binary_data + padding 
        
    else:
        print("[Processing] 패딩을 추가하지 않습니다. (원본 유지)")

    # 3. [선택] 결과를 파일로 저장 ('wb' 모드)
    if output_path:
        with open(output_path, 'wb') as f:
            f.write(binary_data)
        print(f"[Success] 저장 완료: {output_path} (최종 크기: {len(binary_data)} bytes)")
        
    return binary_data

try:
    # MalConvGCT 임포트
    from MalConvGCT_nocat import MalConvGCT
    print("Successfully imported MalConvGCT")

    # 모델 초기화 (README의 파라미터 참고)
    # channels=256, window_size=256, stride=64 설정
    channels = 256
    window_size = 256
    stride = 64
    embd_size = 8
    
    print("Initializing model...")
    model = MalConvGCT(out_size=2, channels=channels, window_size=window_size, stride=stride, embd_size=embd_size)
    
    # 체크포인트 로드
    checkpoint_path = os.path.join(malconv_path, 'malconvGCT_nocat.checkpoint')
    
    if os.path.exists(checkpoint_path):
        print(f"Loading checkpoint from {checkpoint_path}...")
        # Mac M2(Apple Silicon) 호환성을 위해 map_location='cpu' 사용 권장
        checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
        
        # 가중치 로드
        if 'model_state_dict' in checkpoint:
            model.load_state_dict(checkpoint['model_state_dict'], strict=False)
            print("Model weights loaded successfully.")
        else:
            print(f"Warning: 'model_state_dict' not found. Keys: {checkpoint.keys()}")
    else:
        print(f"Warning: Checkpoint file not found at {checkpoint_path}. Running with random weights.")

    # 모델을 평가 모드로 설정
    model.eval()

    # 더미 데이터 생성 및 실행 테스트
    # 입력: 바이트 시퀀스 (0-255 값을 1-256으로 매핑, 0은 패딩)
    # 길이 100,000 바이트 시뮬레이션
    fake_file_len = 100000

    # 원래라면 exe file -> byte seq -> np.formbuffer -> tensor 변환 -> 추후 src에 구현하기 
    dummy_input = torch.randint(low=1, high=257, size=(1, fake_file_len)).long()
    
    print(f"Running inference on dummy input with shape {dummy_input.shape}...")
    

    # --- 사용 예시 ---

    # 1. 아무것도 안 하고 읽기만 하기 (분석용)
    raw_data = manipulate_exe("ZoomInstaller.exe", output_path=None, padding_size=0)

    with torch.no_grad():
        output = model(dummy_input)
        # MalConvGCT returns: (logits, penult, post_conv)
        logits = output[0]
        probabilities = torch.sigmoid(logits)
        
    print("\nExecution Complete!")
    print(f"Logits: {logits}")
    print(f"Probabilities: {probabilities}")

except ImportError as e:
    print(f"Import Error: {e}")
    print("Make sure MalConv2-main is in the path.")
except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()

Successfully imported MalConvGCT
Initializing model...
Loading checkpoint from /Users/wjm/Desktop/2026 프로젝트/Binary-Hunter/models/MalConv2-main/malconvGCT_nocat.checkpoint...
Model weights loaded successfully.
Running inference on dummy input with shape torch.Size([1, 100000])...
[Info] ZoomInstaller.exe 로드 완료. 크기: 44512936 bytes
[Processing] 패딩을 추가하지 않습니다. (원본 유지)

Execution Complete!
Logits: tensor([[ 1.1707, -2.1554]])
Probabilities: tensor([[0.7633, 0.1038]])
